In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# импортируем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')

import warnings
warnings.filterwarnings("ignore")

In [3]:
# загружаем данные
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Work/data.csv')
df.sample(7)

,code_post,year,month,day,heigh,rainy
36627,9403,2015,4,15,92.0,I
8407,9416,2009,8,19,90.0,NaN
30360,9420,2013,1,20,209.0,^I
10092,9518,2009,1,5,131.0,I
22258,9406,2012,11,26,135.0,I
4518,9422,2008,7,5,275.0,NaN
34703,9417,2014,12,9,29.0,NaN


In [4]:
df.shape

(52824, 6)

In [5]:
unique_post = df['code_post'].value_counts()
unique_post

9403    3720
9404    3720
9405    3720
9406    3720
9408    3720
9410    3720
9413    3720
9415    3720
9416    3720
9417    3720
9419    3720
9420    3720
9422    3720
9518    3720
9421     744
Name: code_post, dtype: int64

In [6]:
df = df.drop(df[df['code_post'] == 9421].index)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52080 entries, 0 to 52823
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   code_post  52080 non-null  int64  
 1   year       52080 non-null  int64  
 2   month      52080 non-null  int64  
 3   day        52080 non-null  int64  
 4   heigh      52080 non-null  float64
 5   rainy      30645 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 2.8+ MB


У нас есть пропущенные/отсутствующие данные. В колонке 'rainy' это объяснимо, очевидно, что заполнится нулями, но с высотой уровня - это проблема.

In [8]:
df_1 = df.copy()

In [9]:
def split_and_replace(value):
  if ":" in value or '-' in value:
    parts = value.split(":")
    new_value = parts[0]
    if '-' in new_value:
      new_value = new_value.replace("-", "")
    return new_value
  else:
    return value

df_1['heigh'].astype(str).apply(split_and_replace).replace("-", "") #привели к строковому типу, чтобы использовать apply
df_1['heigh'] = pd.to_numeric(df_1['heigh'], errors='coerce') # преобразовали столбец в числовой формат
df_1['heigh'] = df['heigh'].replace(np.nan, 0) # все наны заменили нулями

In [10]:
df_1['rainy'] = df['rainy'].fillna('unknown') #заполнили пропуски

In [11]:
from datetime import datetime

df_1['date'] = pd.to_datetime(df[['year', 'month', 'day']], format='%d-%m-%Y', dayfirst=True, errors='coerce')

In [12]:
df_1['week'] = df_1['date'].dt.week

In [13]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52080 entries, 0 to 52823
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   code_post  52080 non-null  int64         
 1   year       52080 non-null  int64         
 2   month      52080 non-null  int64         
 3   day        52080 non-null  int64         
 4   heigh      52080 non-null  float64       
 5   rainy      52080 non-null  object        
 6   date       51142 non-null  datetime64[ns]
 7   week       51142 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 3.6+ MB


Сейчас буду кувыркаться - переделывать таблицу

In [14]:
df_9415 = df_1[df['code_post'] == 9415]

In [15]:
df_9415.shape

(3720, 8)

In [16]:
df_9415.head()

,code_post,year,month,day,heigh,rainy,date,week
2604,9415,2008,1,1,964.0,^]I,2008-01-01,1.0
2605,9415,2008,2,1,795.0,^I],2008-02-01,5.0
2606,9415,2008,3,1,709.0,^I],2008-03-01,9.0
2607,9415,2008,4,1,694.0,^I],2008-04-01,14.0
2608,9415,2008,5,1,686.0,_I],2008-05-01,18.0


In [17]:
n = df_9415.duplicated(subset=['date']).sum()
n

66

In [18]:
df_9415['date'].isna().sum()

67

In [19]:
df_9415 = df_9415.dropna(subset=['date'])
df_9415.shape

(3653, 8)

In [20]:
df_1.head()

,code_post,year,month,day,heigh,rainy,date,week
0,9403,2008,1,1,76.0,_I,2008-01-01,1.0
1,9403,2008,2,1,78.0,I,2008-02-01,5.0
2,9403,2008,3,1,78.0,_I,2008-03-01,9.0
3,9403,2008,4,1,79.0,_I,2008-04-01,14.0
4,9403,2008,5,1,87.0,_Z,2008-05-01,18.0


In [ ]:
df.head()

,code_post,year,month,day,heigh,rainy
0,9403,2008,1,1,76,_I
1,9403,2008,2,1,78,I
2,9403,2008,3,1,78,_I
3,9403,2008,4,1,79,_I
4,9403,2008,5,1,87,_Z


In [21]:
df_list = []

for post in df_1['code_post'].unique():
    if post != 9415:
        df_2 = df_1[df_1['code_post'] == post].dropna(subset=['date'])
        df_3 = df_2[['date', 'heigh', 'rainy']] 
        df_list.append(df_3)

In [22]:
df_list

[            date  heigh    rainy
 0     2008-01-01   76.0       _I
 1     2008-02-01   78.0        I
 2     2008-03-01   78.0       _I
 3     2008-04-01   79.0       _I
 4     2008-05-01   87.0       _Z
 ...          ...    ...      ...
 47608 2017-05-31   90.0       ^I
 47610 2017-07-31   58.0        _
 47611 2017-08-31    1.0  unknown
 47613 2017-10-31   53.0  unknown
 47615 2017-12-31   53.0        Z
 
 [3653 rows x 3 columns],
             date  heigh    rainy
 372   2008-01-01  102.0       _I
 373   2008-02-01  108.0       _I
 374   2008-03-01  124.0       _I
 375   2008-04-01  130.0       _I
 376   2008-05-01  140.0       _(
 ...          ...    ...      ...
 47980 2017-05-31  123.0       ^I
 47982 2017-07-31   79.0        _
 47983 2017-08-31    1.0  unknown
 47985 2017-10-31   69.0        _
 47987 2017-12-31   69.0       _I
 
 [3653 rows x 3 columns],
             date  heigh    rainy
 744   2008-01-01  187.0       _I
 745   2008-02-01  193.0       ^I
 746   2008-03-01  192.0  

In [23]:
posts = np.unique(df_1['code_post'][df_1['code_post'] != 9415])
posts

array([9403, 9404, 9405, 9406, 9408, 9410, 9413, 9416, 9417, 9419, 9420,
       9422, 9518])

In [24]:
for i, df in enumerate(df_list):
    df.rename(columns={'heigh': f'heigh_{posts[i]}', 'rainy': f'rainy_{posts[i]}'}, inplace=True)

In [25]:
df_merged = df_list[0]
for df in df_list[1:]:
    df_merged = df_merged.merge(df, on='date', how='outer')

In [26]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        3653 non-null   datetime64[ns]
 1   heigh_9403  3653 non-null   float64       
 2   rainy_9403  3653 non-null   object        
 3   heigh_9404  3653 non-null   float64       
 4   rainy_9404  3653 non-null   object        
 5   heigh_9405  3653 non-null   float64       
 6   rainy_9405  3653 non-null   object        
 7   heigh_9406  3653 non-null   float64       
 8   rainy_9406  3653 non-null   object        
 9   heigh_9408  3653 non-null   float64       
 10  rainy_9408  3653 non-null   object        
 11  heigh_9410  3653 non-null   float64       
 12  rainy_9410  3653 non-null   object        
 13  heigh_9413  3653 non-null   float64       
 14  rainy_9413  3653 non-null   object        
 15  heigh_9416  3653 non-null   float64       
 16  rainy_9416  3653 non-nul

In [27]:
m = df_merged.duplicated(subset=['date']).sum()
m

0

In [28]:
df_merged['date'].isna().sum()

0

In [29]:
df_9415 = df_9415.merge(df_merged, on='date', how='left')

In [ ]:
df_9415.shape

(3653, 34)

In [30]:
df_9415 = df_9415.drop('code_post', axis=1)

In [31]:
df_9415.head()

,year,month,day,heigh,rainy,date,week,heigh_9403,rainy_9403,heigh_9404,...,heigh_9417,rainy_9417,heigh_9419,rainy_9419,heigh_9420,rainy_9420,heigh_9422,rainy_9422,heigh_9518,rainy_9518
0,2008,1,1,964.0,^]I,2008-01-01,1.0,76.0,_I,102.0,...,46.0,_I,315.0,^I,178.0,I,343.0,^Z,125.0,_I
1,2008,2,1,795.0,^I],2008-02-01,5.0,78.0,I,108.0,...,56.0,_I,296.0,^I,177.0,"""I",309.0,^I,134.0,_I
2,2008,3,1,709.0,^I],2008-03-01,9.0,78.0,_I,124.0,...,62.0,_I,295.0,_I,177.0,_I,293.0,^I,142.0,_I
3,2008,4,1,694.0,^I],2008-04-01,14.0,79.0,_I,130.0,...,66.0,_I,298.0,I,178.0,"""I",284.0,^I,160.0,"""I"
4,2008,5,1,686.0,_I],2008-05-01,18.0,87.0,_Z,140.0,...,73.0,_I,298.0,I,179.0,_I,276.0,Z,160.0,_I


In [32]:
df_9415.to_csv('data_9415.csv', index=False)